In [1]:
import pandas as pd
import numpy as np
import pickle
from preprocess_text import *
from bigrams import *
from smoothers import *
from LanguageModels import *

In [2]:
df = pd.read_csv('A1_dataset.csv')

In [3]:
# text = df['TEXT'].to_list()
# def preprocess_text(text):
#     text = lowercase_text(text)
#     text = remove_url_html(text)
#     text = remove_users(text)
#     text = remove_punctuations(text)
#     text = remove_whitespaces(text)
#     text = tokenization(text)
#     text = spelling_correction(text, 'autocorrect_full')
#     text = remove_alphanum(text)
#     return text
# df['preprocessed_text'] = df['TEXT'].progress_apply(preprocess_text)
# preprocess_text = df['preprocessed_text'].to_list()
# df.to_csv('preprocessed_A1.csv', index=False)

In [4]:
df = pd.read_csv('preprocessed_A1.csv')
preprocess_text = df['preprocessed_text'].to_list()
for i in range(len(preprocess_text)):
    preprocess_text[i] = eval(preprocess_text[i])
    

In [5]:
unigram_counts = {}
for sentence in preprocess_text:
    for word in sentence:
        if word in unigram_counts:
            unigram_counts[word] += 1
        else:
            unigram_counts[word] = 1

In [6]:
unigram_probs = {}
scale_factor = sum(unigram_counts.values())
for word in unigram_counts:
    unigram_probs[word] = unigram_counts[word] / scale_factor

In [7]:
len(unigram_counts)

6952

# Saving Bigrams with Smoothing

In [8]:
bigram_wrapper = Bigrams(preprocess_text)
bigrams = bigram_wrapper.get_bigrams()
vocab_len = bigram_wrapper.vocab_len

with open('bigram_wrapper.pickle', 'wb') as f:
    pickle.dump(bigram_wrapper, f)

In [9]:
# load bigram_wrapper
# with open('bigram_wrapper.pickle', 'rb') as f:
#     bigram_wrapper = pickle.load(f)

# bigrams = bigram_wrapper.get_bigrams()
# vocab_len = bigram_wrapper.vocab_len

In [10]:
smoother = LMSmoothers(bigrams, vocab_len, unigram_counts=unigram_counts, unigrams_probs=unigram_probs)
laplace_smoothed_bigrams = smoother.laplace()
# add_5_smoothed_bigrams = smoother.add_k(5)
# add_5_with_unigram_prior_smoothed_bigrams = smoother.add_k_with_unigram_prior(5)

In [11]:
with open('laplace_smoothed_bigrams.pickle', 'wb') as handle:
    pickle.dump(laplace_smoothed_bigrams, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('add_5_smoothed_bigrams.pickle', 'wb') as handle:
#     pickle.dump(add_5_smoothed_bigrams, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('add_5_with_unigram_prior_smoothed_bigrams.pickle', 'wb') as handle:
#     pickle.dump(add_5_with_unigram_prior_smoothed_bigrams, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Creating BiGram Models

In [12]:
with open('laplace_smoothed_bigrams.pickle', 'rb') as handle:
    laplace_smoothed_bigrams = pickle.load(handle)

# with open('add_5_smoothed_bigrams.pickle', 'rb') as handle:
#     add_5_smoothed_bigrams = pickle.load(handle)

# with open('add_5_with_unigram_prior_smoothed_bigrams.pickle', 'rb') as handle:
#     add_5_with_unigram_prior_smoothed_bigrams = pickle.load(handle)

In [13]:
lm = LanguageModel(laplace_smoothed_bigrams, unigram_probs, "vader", False, False, True, repetition_penalty=0.2)

In [14]:
# Load Negative Prompts
with open('neg_250_vader.txt', 'rb') as handle:
    neg_prompts = handle.readlines()

for i in range(len(neg_prompts)):
    neg_prompts[i] = neg_prompts[i].decode('utf-8').strip()

In [15]:
lm = LanguageModel(laplace_smoothed_bigrams, unigram_probs, "vader", False, False, True, repetition_penalty=0.2)
lm.generate_text(['hell'], -1, 7)

['hell', 'yeah', 'ive', 'been', 'a', 'little', 'bit']

In [16]:
lm.computePerplexity(lm.generate_text(['hell'], -1, 7))

641.2602032666833

In [17]:
# clear the file
open('neg_gen_only_ext.txt', 'w').close()

In [18]:
def append_to_file(filename, text1, text2):
    with open(filename, 'a') as f:
        f.write(text1 + " " + text2 + '\n')

In [19]:
ls_neg_gen = []
lm = LanguageModel(laplace_smoothed_bigrams, unigram_probs, "vader", False, False, True, repetition_penalty=0.2)
for p in neg_prompts:
    sentence_neg = lm.generate_text([p], sentiment=-1, length=10)
    print("Negative: {}".format(sentence_neg))
    ppl = lm.computePerplexity(sentence_neg)
    print("Negative Perplexity: {}".format(ppl))
    ls_neg_gen.append((sentence_neg, ppl))
    append_to_file('neg_gen_only_ext.txt', " ".join(sentence_neg), str(ppl))

Negative: ['raping', 'hell', 'is', 'a', 'great', 'weekend', 'to', 'be', 'a', 'few']
Negative Perplexity: 829.9808557932836
Negative: ['slavery', 'raping', 'tragedy', 'hell', 'is', 'the', 'new', 'iphone', 'from', 'the']
Negative Perplexity: 2086.6723835529174
Negative: ['kill', 'raping', 'slavery', 'killed', 'the', 'best', 'of', 'my', 'mom', 'got']
Negative Perplexity: 2014.3949155207056
Negative: ['hell', 'everybody', 'is', 'the', 'new', 'followers', 'a', 'good', 'luck', 'with']
Negative Perplexity: 1225.1286728660962
Negative: ['killed', 'tea', 'yet', 'again', 'and', 'i', 'have', 'to', 'get', 'my']
Negative Perplexity: 708.6973256261734
Negative: ['killing', 'me', 'too', 'i', 'have', 'to', 'the', 'day', 'with', 'my']
Negative Perplexity: 420.33889155784993
Negative: ['evil', 'and', 'i', 'am', 'not', 'the', 'same', 'name', 'with', 'me']
Negative Perplexity: 824.5222126215607
Negative: ['horrific', 'slavery', 'hell', 'yeah', 'i', 'have', 'to', 'the', 'day', 'to']
Negative Perplexity: 77

In [20]:
# save generated sentences
with open('ls_neg_gen_vader_thread.pickle', 'wb') as handle:
    pickle.dump(ls_neg_gen, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
ls_neg_gen = []
lm = LanguageModel(laplace_smoothed_bigrams, unigram_probs, "vader", True, False, True, repetition_penalty=0.2, sentiment_scale_factor=1e-1)
for p in neg_prompts:
    sentence_neg = lm.generate_text([p], sentiment=-1, length=10)
    print("Negative: {}".format(sentence_neg))
    ppl = lm.computePerplexity(sentence_neg)
    print("Negative Perplexity: {}".format(ppl))
    ls_neg_gen.append((sentence_neg, ppl))
    append_to_file('neg_gen_add_numerator.txt', " ".join(sentence_neg), str(ppl))

['raping', 'evil', 'killed', 'hell', 'horrific', 'tragedy', 'shittiest', 'kill', 'killing', 'dead']
Negative: ['raping', 'evil', 'killed', 'hell', 'horrific', 'tragedy', 'shittiest', 'kill', 'killing', 'dead']
Negative Perplexity: 6956.29877791198
['slavery', 'killed', 'horrific', 'tragedy', 'hell', 'killing', 'shittiest', 'evil', 'dead', 'raping']
Negative: ['slavery', 'killed', 'horrific', 'tragedy', 'hell', 'killing', 'shittiest', 'evil', 'dead', 'raping']
Negative Perplexity: 6957.597555138508
['kill', 'slavery', 'evil', 'tragedy', 'hell', 'killing', 'shittiest', 'horrific', 'dead', 'raping']
Negative: ['kill', 'slavery', 'evil', 'tragedy', 'hell', 'killing', 'shittiest', 'horrific', 'dead', 'raping']
Negative Perplexity: 6957.397569579159
['hell', 'killed', 'slavery', 'raping', 'horrific', 'kill', 'devastating', 'whore', 'shittiest', 'killing']
Negative: ['hell', 'killed', 'slavery', 'raping', 'horrific', 'kill', 'devastating', 'whore', 'shittiest', 'killing']
Negative Perplexity:

In [25]:
ls_neg_gen = []
lm = LanguageModel(laplace_smoothed_bigrams, unigram_probs, "vader", True, False, True, repetition_penalty=0.2)
for p in neg_prompts:
    sentence_neg = lm.generate_text([p], sentiment=-1, length=10)
    print(sentence_neg)
    print("Negative: {}".format(sentence_neg))
    ppl = lm.computePerplexity(sentence_neg)
    print("Negative Perplexity: {}".format(ppl))
    ls_neg_gen.append((sentence_neg, ppl))
    append_to_file('neg_gen_mul_numerator.txt', " ".join(sentence_neg), str(ppl))

['raping', 'evil', 'hell', 'killed', 'tragedy', 'killing', 'shittiest', 'whore', 'slavery', 'kill']
Negative: ['raping', 'evil', 'hell', 'killed', 'tragedy', 'killing', 'shittiest', 'whore', 'slavery', 'kill']
Negative Perplexity: 6955.898714709328
['slavery', 'killed', 'horrific', 'kill', 'evil', 'tragedy', 'raping', 'whore', 'dead', 'hell']
Negative: ['slavery', 'killed', 'horrific', 'kill', 'evil', 'tragedy', 'raping', 'whore', 'dead', 'hell']
Negative Perplexity: 6955.998132896965
['kill', 'killed', 'horrific', 'hell', 'slavery', 'shittiest', 'tragedy', 'devastating', 'raping', 'hated']
Negative: ['kill', 'killed', 'horrific', 'hell', 'slavery', 'shittiest', 'tragedy', 'devastating', 'raping', 'hated']
Negative Perplexity: 6955.39879020979
['hell', 'slavery', 'evil', 'horrific', 'killed', 'shittiest', 'raping', 'whore', 'dead', 'killing']
Negative: ['hell', 'slavery', 'evil', 'horrific', 'killed', 'shittiest', 'raping', 'whore', 'dead', 'killing']
Negative Perplexity: 6956.69739940

Traceback (most recent call last):
  File "c:\Kyode\clg\nlp\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\gener\AppData\Local\Temp\ipykernel_14048\1431889808.py", line 4, in <module>
    sentence_neg = lm.generate_text([p], sentiment=-1, length=10)
  File "c:\Kyode\clg\nlp\A2\LanguageModels.py", line 45, in generate_text
    next_word = self.get_next_word(last_word, sentiment, generated_so_far, sentence_so_far = prompt)
  File "c:\Kyode\clg\nlp\A2\LanguageModels.py", line -1, in get_next_word
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Kyode\clg\nlp\venv\lib\site-packages\IPython\core\interactiveshell.py", line 1997, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Kyode\clg\nlp\venv\lib\site-packages\IPython\core\ultratb.py", line 1112, in structured_traceback
  